# Task 2 #
==========

In [1]:
# Импортируем библиотеки
import sqlite3
import pandas as pd

## Создай соединение с базой данных с помощью библиотеки sqlite3. ##
----------

In [2]:
# Создаем соединение с базой данных
conn = sqlite3.connect("../data/checking-logs.sqlite")

## Получи схему таблицы checker. ##
------

In [3]:
# Получаем схему таблицы checker
checker_schema = pd.io.sql.read_sql('PRAGMA table_info(checker);', conn)
print(checker_schema)

   cid       name       type  notnull dflt_value  pk
0    0      index    INTEGER        0       None   0
1    1     status       TEXT        0       None   0
2    2    success    INTEGER        0       None   0
3    3  timestamp  TIMESTAMP        0       None   0
4    4  numTrials    INTEGER        0       None   0
5    5    labname       TEXT        0       None   0
6    6        uid       TEXT        0       None   0


## Получи только первые 10 строк таблицы checker, чтобы проверить, как она выглядит. ##
-----

In [4]:
# Получаем только первые 10 строк таблицы checker
checker_first_10 = pd.read_sql('SELECT * FROM checker LIMIT 10;', conn)
print(checker_first_10)

   index    status  success                   timestamp  numTrials   labname  \
0      0  checking        0  2020-04-16 21:12:50.740474          5      None   
1      1     ready        0  2020-04-16 21:12:54.708365          5  code_rvw   
2      2  checking        0  2020-04-16 21:46:47.769088          7      None   
3      3     ready        0  2020-04-16 21:46:48.121217          7     lab02   
4      4  checking        0  2020-04-16 21:53:01.862637          6  code_rvw   
5      5     ready        0  2020-04-16 21:53:05.373389          6  code_rvw   
6      6  checking        0  2020-04-17 05:18:51.965864          1      None   
7      7     ready        0  2020-04-17 05:19:02.744528          1  project1   
8      8  checking        0  2020-04-17 05:22:35.249331          2  project1   
9      9     ready        1  2020-04-17 05:22:45.549397          2  project1   

       uid  
0  admin_1  
1  admin_1  
2  admin_1  
3  admin_1  
4  admin_1  
5  admin_1  
6     None  
7   user_4  
8 

## Подсчитай, сколько строк удовлетворяют следующим условиям, используя только один запрос с любым количеством подзапросов ##
-----

* Подсчитай количество строк из таблицы pageviews, но только с пользователями из таблицы checker, у которых:
    - status = 'ready' (мы не хотим анализировать логи со статусом checking);
    - numTrials = 1 (мы хотим проанализировать только первые коммиты, поскольку только они информируют о том, когда студент начал работать над лабораторным заданием);
    - названия лабораторных заданий должны быть только из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1. Только они были активными во время проведения эксперимента.
* Сохрани их в датафрейме checkers в столбце cnt. Датафрейм будет представлять собой только одну ячейку.

In [5]:
# Подсчитываем количество строк из таблицы pageviews с учетом условий задания
cnt_query = '''
SELECT COUNT(*) as cnt FROM pageviews
WHERE uid IN 
    (SELECT DISTINCT uid FROM checker 
     WHERE status = 'ready' AND numTrials = 1 
     AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    );
'''

### Для проверки

In [6]:
checkers = pd.read_sql(cnt_query, conn)
print(checkers)

   cnt
0  985


## Закрой соединение с базой данных. ##
------

In [7]:
conn.close()